In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
def clean_text(text):
    if text:
        text = f'{text}'
        text = text.lower()
        text = text.split()
        ps = PorterStemmer()
        text = [ps.stem(word) for word in text if text not in stopwords.words('english')]
        text = ' '.join(text)
        return text

In [ ]:
#Import dataset 5 (185 samples)
X  = []
y = []

with open("/content/gdrive/MyDrive/special_fake_news.txt",'r',encoding = 'latin-1') as file:
    texts = file.read().split('\n')
    for text in texts:
        if text:
            X.append(clean_text(text))
            y.append(0)

with open("/content/gdrive/MyDrive/special_real_news.txt",'r',encoding = 'latin-1') as file:
    texts = file.read().split('\n')
    for text in texts:
        if text:
            X.append(clean_text(text))
            y.append(1)



In [ ]:
#Importing dataset 1 (First 5000 samples)
data = pd.read_csv("/content/gdrive/MyDrive/train.csv")

X = []
y = []
for i in range(5000):
  X.append(clean_text(data['title'][i]))
  y.append(data['label'][i])

In [ ]:
#Import dataset 2 (First 5000 samples)
data = pd.read_csv("/content/gdrive/MyDrive/fake_or_real_news.csv")

X = []
y = []

for i in range(5000):
    X.append(clean_text(data['title'][i]))
    if data['label'][i] == 'FAKE':
        y.append(0)
    else:
        y.append(1)

In [ ]:
#Import dataset 3
fake_data = pd.read_csv("/content/gdrive/MyDrive/Fake.csv",engine = 'python')
true_data = pd.read_csv("/content/gdrive/MyDrive/True.csv",engine = 'python')

#Take 2500 samples from true data , 4064 samples from fake data
X = []
y = []
for i in range(4064):
    X.append(clean_text(fake_data['title'][i]))
    y.append(0)
for i in range(2500):
    X.append(clean_text(true_data['title'][i]))
    y.append(1)

In [ ]:
#Import dataset 4 (7950 samples)
import sys
import csv

csv.field_size_limit(sys.maxsize)
fake_data = pd.read_csv("/content/gdrive/MyDrive/Fake.csv",engine = 'python')
fake_data_2 = pd.read_csv("/content/gdrive/MyDrive/fake2.csv",engine = 'python')
true_data = pd.read_csv("/content/gdrive/MyDrive/True.csv",engine = 'python')


X = []
y = []
for i in range(1620):
    X.append(clean_text(fake_data['title'][i]))
    y.append(0)
for i in range(2880):
    X.append(clean_text(fake_data_2['title'][i]))
    y.append(0)
for i in range(3450):
    X.append(clean_text(true_data['title'][i]))
    y.append(1)

In [ ]:
#Import dataset 6 
data = pd.read_csv("/content/gdrive/MyDrive/data.csv")
X = []
y = data['Label']
for i in range(len(data)):
    X.append(clean_text(data['Headline'][i]))

In [ ]:
#Import dataset 7 
X = []
y = []

#Get 2000 samples from dataset 6 
data = pd.read_csv("/content/gdrive/MyDrive/data.csv")
for i in range(2000):
    X.append(clean_text(data['Headline'][i]))
    y.append(data['Label'][i])

#Get all from dataset 5
with open("/content/gdrive/MyDrive/special_fake_news.txt",'r',encoding = 'latin-1') as file:
    texts = file.read().split('\n')
    for text in texts:
        if text:
            X.append(clean_text(text))
            y.append(0)

with open("/content/gdrive/MyDrive/special_real_news.txt",'r',encoding = 'latin-1') as file:
    texts = file.read().split('\n')
    for text in texts:
        if text:
            X.append(clean_text(text))
            y.append(1)

#Get 2000 samples from dataset 1
data = pd.read_csv("/content/gdrive/MyDrive/train.csv")
for i in range(5000):
  X.append(clean_text(data['title'][i]))
  y.append(data['label'][i])

#Get 3000 samples from dataset 3
fake_data = pd.read_csv("/content/gdrive/MyDrive/Fake.csv",engine = 'python')
true_data = pd.read_csv("/content/gdrive/MyDrive/True.csv",engine = 'python')

#Take 1100 samples from true data , 1900 samples from fake data
for i in range(1100):
    X.append(clean_text(fake_data['title'][i]))
    y.append(0)
for i in range(1900):
    X.append(clean_text(true_data['title'][i]))
    y.append(1)

In [ ]:
#Features selection
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X = cv.fit_transform(X).toarray()

In [ ]:
#Split test set
from sklearn.model_selection import train_test_split 
X_train , X_test , y_train , y_test = train_test_split(X,y, test_size = 0.30,random_state = 0)

In [ ]:
#Import necessary libraries for classification
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier

In [ ]:
#Build and fit model to dataset
classifier = VotingClassifier(estimators = [('naive_bayes',MultinomialNB()),
                                            ('logistic_regression',LogisticRegression(n_jobs = -1 , random_state = 0)),
                                            #('knn',KNeighborsClassifier(n_neighbors=10,n_jobs = -1)),
                                            ('svm',SVC(random_state = 0))
                                            ],
                                            voting = 'hard',
                                            n_jobs = -1)

classifier = classifier.fit(X_train,y_train)

In [ ]:
y_pred = classifier.predict(X_test)

In [ ]:
#Making confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test , y_pred)

In [ ]:
def calculate_accuracy(cm):
    tn = cm[0][0]
    tp = cm[1][1]
    fp = cm[0][1]
    fn = cm[1][0]
        
    accuracy = (tn + tp) / (tn + tp + fp + fn)
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    f1 = 2*(precision*recall)/(precision + recall)
    average = (accuracy + precision + recall + f1) /4 
    return [accuracy , precision , recall , f1 , average]

In [ ]:
acc = calculate_accuracy(cm)

In [ ]:
acc

[0.8459512837393022,
 0.8142292490118577,
 0.9120809614168248,
 0.8603818615751789,
 0.858160838935791]

In [ ]:
len(X)

10125